In [ ]:
# default_exp solar

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Solar

> Access data and analysis services that provide access to solar resource data and NREL models using a python API.

> NOTE: In order to use the NREL developer API, you will need an API key from NREL. You can get one [here](https://developer.nrel.gov/signup/). The process is simple and only requires your name and email where you will receive the API key. 

In [ ]:
#hide
from nbdev.showdoc import *

# SolarResourceData

In [ ]:
#export
from nrel_dev_api.core import _GetPostRequest

# TODO - add attributes in docstring, add checks for data inputs
class SolarResourceData(_GetPostRequest):
    """Returns various types of solar data for a location as a dictionary. 
    The service from NREL currently returns data for 
    average Direct Normal Irradiance, average Global Horizontal Irradiance, 
    and average Tilt at Latitude.
    
    Attributes:
        api_key: NREL Developer API key (https://developer.nrel.gov/signup/).
        lat: Latitude of the location.
        lon: Longitude of the location.
        address: Address of the location to use. Required if lat/lon not specified.
    """
    
    QUERY_URL = "/api/solar/solar_resource/v1.json"
    
    def __init__(self, api_key, lat=None, lon=None, address=None):
        
        super().__init__()
        
        self._params.update({"api_key": api_key})
        
        # if address is not specified latitude and longitude must be specified
        if not address:
            self._params.update({"lat" : lat, "lon" : lon})
        else:
            self._params.update({"address": address})
            
        # complete response as a dict
        self.response = self._get()
        
        # only the outputs
        self.outputs = self.response["outputs"]
        
        # get the inputs provided
        self.inputs = self.response["inputs"]

An example to get solar resource data - average Direct Normal Irradiance, average Global Horizontal Irradiance, and average tilt - from NREL

In [ ]:
#hide
NREL_API_KEY = "DEMO_KEY"

In [ ]:
solar_resource_data = SolarResourceData(api_key=NREL_API_KEY, lat=40, lon=-105)

Outputs for solar resource data

In [ ]:
solar_resource_data.outputs

{'avg_dni': {'annual': 6.06,
  'monthly': {'jan': 5.0,
   'feb': 5.34,
   'mar': 5.94,
   'apr': 6.11,
   'may': 6.36,
   'jun': 7.43,
   'jul': 7.48,
   'aug': 6.65,
   'sep': 6.81,
   'oct': 5.82,
   'nov': 5.11,
   'dec': 4.67}},
 'avg_ghi': {'annual': 4.81,
  'monthly': {'jan': 2.5,
   'feb': 3.43,
   'mar': 4.69,
   'apr': 5.69,
   'may': 6.6,
   'jun': 7.25,
   'jul': 7.14,
   'aug': 6.24,
   'sep': 5.35,
   'oct': 3.85,
   'nov': 2.75,
   'dec': 2.19}},
 'avg_lat_tilt': {'annual': 5.82,
  'monthly': {'jan': 4.79,
   'feb': 5.4,
   'mar': 6.07,
   'apr': 6.11,
   'may': 6.25,
   'jun': 6.47,
   'jul': 6.58,
   'aug': 6.44,
   'sep': 6.53,
   'oct': 5.71,
   'nov': 4.99,
   'dec': 4.47}}}

Use address to access the solar resource data

In [ ]:
address = "Seattle, WA"

solar_resource_data = SolarResourceData(api_key=NREL_API_KEY, address=address)

Get the complete response as a dictionary

In [ ]:
solar_resource_data.response

{'version': '1.0.0',
 'warnings': [],
 'errors': [],
 'metadata': {'sources': ['Perez-SUNY/NREL, 2012']},
 'inputs': {'address': 'Seattle, WA'},
 'outputs': {'avg_dni': {'annual': 3.5,
   'monthly': {'jan': 1.19,
    'feb': 2.73,
    'mar': 2.16,
    'apr': 3.76,
    'may': 4.21,
    'jun': 5.14,
    'jul': 6.66,
    'aug': 5.97,
    'sep': 5.12,
    'oct': 2.51,
    'nov': 1.34,
    'dec': 1.21}},
  'avg_ghi': {'annual': 3.46,
   'monthly': {'jan': 0.98,
    'feb': 1.94,
    'mar': 2.75,
    'apr': 4.51,
    'may': 5.34,
    'jun': 5.99,
    'jul': 6.38,
    'aug': 5.48,
    'sep': 4.03,
    'oct': 2.11,
    'nov': 1.12,
    'dec': 0.84}},
  'avg_lat_tilt': {'annual': 3.98,
   'monthly': {'jan': 1.73,
    'feb': 3.25,
    'mar': 3.43,
    'apr': 4.95,
    'may': 5.13,
    'jun': 5.43,
    'jul': 5.98,
    'aug': 5.87,
    'sep': 5.23,
    'oct': 3.18,
    'nov': 1.91,
    'dec': 1.65}}}}

Complete response.

# PVWatts V6

In [ ]:
#export

# TODO - add attributes in docstring, add checks for data inputs
class PVWattsV6(_GetPostRequest):
    """Estimate the energy production of grid-connected photovoltaic (PV) energy systems
    using NREL's PVWatts API based on a few simple inputs.
    """
    
    QUERY_URL = "/api/pvwatts/v6.json"
    
    def __init__(self,
                 api_key,
                 system_capacity,
                 module_type,
                 losses,
                 array_type,
                 tilt,
                 azimuth,
                 lat=None,
                 lon=None,
                 address=None,
                 file_id=None,
                 dataset="nsrdb",
                 radius=100,
                 timeframe="monthly",
                 dc_ac_ratio=1.2,
                 gcr=0.4,
                 inv_eff=96,
                ):
        
        super().__init__()
        
        self._params.update({
            "api_key" : api_key,
            "system_capacity" : system_capacity,
            "module_type" : module_type,
            "losses" : losses,
            "array_type" : array_type,
            "tilt" : tilt,
            "azimuth" : azimuth,
        })
        
        # only one of lat/lon, file_id, address needs to be specified
        if not address and not file_id:
            self._params.update({"lat" : lat, "lon" : lon})
        
        if not file_id and not lat and not lon:
            self._params.update({"address" : address})
            
        if not address and not lat and lon:
            self._params.update({"file_id" : file_id})
        
        # if file_id is specified, dataset info is not required
        if not file_id:
            self._params.update({"dataset" : dataset})
        
        self._params.update({
            "radius" : radius,
            "timeframe" : timeframe,
            "dc_ac_ratio" : dc_ac_ratio,
            "gcr" : gcr,
            "inv_eff" : inv_eff
        })
        
        # get the complete response
        self.response = self._get()
        
        # only the outputs category
        self.outputs = self.response["outputs"]
        
        # station info for the specified lat/lon
        self.station_info = self.response["station_info"]
        
        # get the inputs provided
        self.inputs = self.response["inputs"]

Example to use PVWattsV6 for a given set of location and system inputs.

In [ ]:
pvwatts_v6 = PVWattsV6(api_key=NREL_API_KEY,
                       system_capacity=4,
                       lat=40,
                       lon=-105,
                       azimuth=180,
                       tilt=40,
                       array_type=1,
                       module_type=1,
                       losses=10
                      )

Output from PVWatts

In [ ]:
pvwatts_v6.outputs

{'ac_monthly': [474.4326171875,
  484.3903503417969,
  595.7704467773438,
  592.0599365234375,
  591.2662353515625,
  589.3538208007812,
  583.2352905273438,
  586.4593505859375,
  584.8131713867188,
  561.72314453125,
  486.1260375976562,
  445.6881713867188],
 'poa_monthly': [141.4809417724609,
  145.5711975097656,
  184.7876434326172,
  181.5513763427734,
  186.4280853271484,
  190.5132904052734,
  188.7499694824219,
  190.5398101806641,
  188.2213134765625,
  175.4444122314453,
  146.3170471191406,
  131.1568298339844],
 'solrad_monthly': [4.563901424407959,
  5.198971271514893,
  5.960891723632812,
  6.051712512969971,
  6.013809204101562,
  6.350442886352539,
  6.088708877563477,
  6.146445274353027,
  6.274043560028076,
  5.659497261047363,
  4.877234935760498,
  4.230865478515625],
 'dc_monthly': [497.9421081542969,
  511.1524963378906,
  634.3223266601562,
  623.7313842773438,
  617.8812866210938,
  615.4278564453125,
  609.3965454101562,
  612.7506713867188,
  610.55841064453

Station info for the latitude/longitude specified

In [ ]:
pvwatts_v6.station_info

{'lat': 40.0099983215332,
 'lon': -105.0199966430664,
 'elev': 1581.839965820312,
 'tz': -7.0,
 'location': 'None',
 'city': '',
 'state': 'Colorado',
 'solar_resource_file': 'W10502N4001.csv',
 'distance': 2029}

Get the complete response

In [ ]:
pvwatts_v6.response

{'inputs': {'system_capacity': '4',
  'module_type': '1',
  'losses': '10',
  'array_type': '1',
  'tilt': '40',
  'azimuth': '180',
  'lat': '40',
  'lon': '-105',
  'dataset': 'nsrdb',
  'radius': '100',
  'timeframe': 'monthly',
  'dc_ac_ratio': '1.2',
  'gcr': '0.4',
  'inv_eff': '96'},
 'errors': [],
 'warnings': [],
 'version': '1.0.2',
 'ssc_info': {'version': 45,
  'build': 'Linux 64 bit GNU/C++ Jul  7 2015 14:24:09'},
 'station_info': {'lat': 40.0099983215332,
  'lon': -105.0199966430664,
  'elev': 1581.839965820312,
  'tz': -7.0,
  'location': 'None',
  'city': '',
  'state': 'Colorado',
  'solar_resource_file': 'W10502N4001.csv',
  'distance': 2029},
 'outputs': {'ac_monthly': [474.4326171875,
   484.3903503417969,
   595.7704467773438,
   592.0599365234375,
   591.2662353515625,
   589.3538208007812,
   583.2352905273438,
   586.4593505859375,
   584.8131713867188,
   561.72314453125,
   486.1260375976562,
   445.6881713867188],
  'poa_monthly': [141.4809417724609,
   145.5

# SolarDatasetQuery

In [ ]:
#export

# TODO - add attributes in docstring, add checks for data inputs
class SolarDatasetQuery(_GetPostRequest):
    
    """Returns information on the closest climate data for a location.
    """
    
    QUERY_URL = "/api/solar/data_query/v1.json"
    
    def __init__(self,
                 api_key,
                 lat=None,
                 lon=None,
                 address=None,
                 radius=100,
                 return_all_stations=False,
                ):
        
        super().__init__()
        
        self._params.update({
            "api_key" : api_key,
            "radius" : radius,
            "all" : 0 if return_all_stations is False else 1,
        })
        
        # if address is not specified latitude and longitude must be specified
        if not address:
            self._params.update({"lat" : lat, "lon" : lon})
        else:
            self._params.update({"address": address})
        
        self.response = self._get()
        
        # only outputs
        self.outputs = self.response["outputs"]
        
        # get the inputs provided
        self.inputs = self.response["inputs"]

Information on climate data for the location closest to the specified one.

In [ ]:
solar_dataset_query = SolarDatasetQuery(api_key=NREL_API_KEY,
                                        lat=40,
                                        lon=-105,
                                       )

Outputs for the solar dataset

In [ ]:
solar_dataset_query.outputs

{'tmy2': {'id': '0-94018',
  'city': 'BOULDER',
  'state': 'COLORADO',
  'timezone': -7,
  'lat': 40.13,
  'lon': -105.24,
  'elevation': 1689,
  'distance': 24977},
 'tmy3': {'id': '1-724699',
  'city': 'BROOMFIELD/JEFFCO [BOULDER - SURFRAD]',
  'state': 'COLORADO',
  'timezone': -7,
  'lat': 40.13,
  'lon': -105.24,
  'elevation': 1689,
  'distance': 24977},
 'intl': None,
 'nsrdb': {'id': '3-W105N040-W10502N4001',
  'city': '',
  'state': 'Colorado',
  'country': None,
  'lat': 40.01,
  'lon': -105.02,
  'distance': 2029,
  'timezone': -7,
  'elevation': 1582,
  'resolution': 4}}

Get information about all the stations within a specified radius

In [ ]:
solar_dataset_query = SolarDatasetQuery(api_key=NREL_API_KEY,
                                        lat=40,
                                        lon=-105,
                                        radius=100,
                                        return_all_stations=True
                                       )

solar_dataset_query.outputs

/Users/stlp/Documents/Github/nrel_dev_api/nrel_dev_api/core.py:30: UserWarning: ['No intl data found for lat=40.0 lon=-105.0']
  warnings.warn(message=content["warnings"], category=UserWarning)


{'tmy2': {'id': '0-94018',
  'city': 'BOULDER',
  'state': 'COLORADO',
  'timezone': -7,
  'lat': 40.13,
  'lon': -105.24,
  'elevation': 1689,
  'distance': 24977},
 'tmy3': {'id': '1-724699',
  'city': 'BROOMFIELD/JEFFCO [BOULDER - SURFRAD]',
  'state': 'COLORADO',
  'timezone': -7,
  'lat': 40.13,
  'lon': -105.24,
  'elevation': 1689,
  'distance': 24977},
 'intl': None,
 'nsrdb': {'id': '3-W105N040-W10502N4001',
  'city': '',
  'state': 'Colorado',
  'country': None,
  'lat': 40.01,
  'lon': -105.02,
  'distance': 2029,
  'timezone': -7,
  'elevation': 1582,
  'resolution': 4},
 'all_stations': [{'id': '3-W105N040-W10502N4001',
   'city': '',
   'state': 'Colorado',
   'country': None,
   'lat': 40.01,
   'lon': -105.02,
   'distance': 2029,
   'timezone': -7,
   'elevation': 1582,
   'resolution': 4,
   'dataset': 'nsrdb'},
  {'id': '0-94018',
   'city': 'BOULDER',
   'state': 'COLORADO',
   'timezone': -7,
   'lat': 40.13,
   'lon': -105.24,
   'elevation': 1689,
   'distance': 